In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime

engine = create_engine(f'sqlite:///../data/algoritmo_comisiones_odoo.db')

In [2]:
with engine.connect() as conn, conn.begin():  
    dfven_Original = pd.read_sql_table('ven_Original', conn)

engine.dispose()

In [3]:
lista_vendedoras_ce = ['Mayra Angelica Parada Manjarrez','Irma Carvajal Flores','Brenda Luz Acosta Lopez','Yamilet B. Blanco Salas']

In [7]:
def almacen(Almacén: str) -> str:
    if 'Almacén 1' in Almacén:
        return 1
    elif 'Almacén 2' in Almacén:
        return 2
    else:
        return Almacén

def CE_Piso(Vendedor: str) -> str:
    if Vendedor in lista_vendedoras_ce:
        return 'CE'
    else:
        return 'Piso'

dfven = dfven_Original.copy()
dfven['Almacén'] = dfven['Almacén'].apply(almacen)
dfven['CE_Piso'] = dfven['Vendedor'].apply(CE_Piso)


¿Cuál es la órden de venta de Paty dónde más vendió y cuánto?

In [184]:
vendedora = 'Patricia Flores Pantaleón'

paty = dfven[dfven['Vendedor'] == vendedora]

wep = paty[['Referencia de la orden','Subtotal']].groupby('Referencia de la orden').sum().sort_values(by=['Subtotal'],ascending=False).iloc[0]

f"Paty vendió ${wep['Subtotal']:,.2f} en la orden {wep.name}"

'Paty vendió $26,189.54 en la orden S05321'

¿Cuáles son los 5 artículos que más vende Paty?

In [111]:
wep = paty[['Producto/Referencia interna','Producto','Cantidad']].groupby('Producto/Referencia interna').sum().sort_values(by=['Cantidad'],ascending=False).iloc[0:5]
wep

,Producto,Cantidad
Producto/Referencia interna,,
13154,"[13154] Pija Autorr. C/Rondana 8*2"" Galv. (Kla...",4700.0
9917,[9917] Pija C/Pl 6*5/8 P/Bisagra (Fhsb658) *99...,2217.0
6528,"[6528] Pija Autorr. C/Rondana 8""*1/2"" Galv. *6...",1500.0
10179,"[10179] Taq. Plastico El-Pro 1/4"" Blanco TQ-01...",1220.0
49507,[49507] Cubrebocas 3 capas quirúrgico Negro (P...,500.0


In [112]:
wep = paty[['Producto/Referencia interna','Producto','Subtotal']].groupby('Producto/Referencia interna').sum().sort_values(by=['Subtotal'],ascending=False).iloc[0:5]
wep

,Producto,Subtotal
Producto/Referencia interna,,
0,[0] Nuevo *0*[0] Nuevo *0*[0] Nuevo *0*[0] Nue...,65426.65
33661,"[33661] Masking Tape 3M 1"" #2090 *33661*[3366...",22872.18
1846,[1846] Termometro digital para alimentos Taylo...,12607.65
3034,[3034] Lija Ban. 4*24 Gd. 80 *3034*[3034] Lija...,7949.76
24610,[24610] Martillo sds plus 1* 800w M8701B-1 MAK...,6398.53


¿Cuáles son los mejores 3 clientes de Paty? 1ro por cantidad de productos vendidos, 2do por subtotal de venta y el último por cantidad de OV en el mes.

In [167]:
wep = paty[['Cliente','Cantidad','Subtotal','Referencia de la orden']].groupby('Cliente').agg({'Referencia de la orden':['count',],'Cantidad':['sum',],'Subtotal':['sum',]})

cov = wep[('Referencia de la orden', 'count')].sort_values(ascending=False)
cov_name = cov.keys()[0]
cov_cant = cov.loc[cov_name]

ccant = wep[('Cantidad', 'sum')].sort_values(ascending=False)
ccant_name = ccant.keys()[0]
ccant_cant = ccant.loc[ccant_name]

csub = wep[('Subtotal', 'sum')].sort_values(ascending=False)
csub_name = csub.keys()[0]
csub_cant = csub.loc[csub_name]


In [187]:
print(f'La vendedora {vendedora} tiene a sus 3 mejores clientes de la siguiente manera:')
print(f'\tEl cliente {cov_name} es el cliente que más órdenes de venta tiene, siendo un total de {cov_cant:,.2f} órdenes')
print(f'\tEl cliente {ccant_name} es el cliente que más cantidad de producto, siendo un total de {ccant_cant:,.2f} artículos vendidos')
print(f'\tEl cliente {csub_name} es el cliente que más facturó, siendo un total de ${csub_cant:,.2f} pesos en el mes de enero')

La vendedora Patricia Flores Pantaleón tiene a sus 3 mejores clientes de la siguiente manera:
	El cliente Mostrador es el cliente que más órdenes de venta tiene, siendo un total de 740.00 órdenes
	El cliente Mostrador es el cliente que más cantidad de producto, siendo un total de 5,743.25 artículos vendidos
	El cliente Mostrador es el cliente que más facturó, siendo un total de $126,723.03 pesos en el mes de enero
